# Sensitivity with systematics

#### Exercise
* Implement a response in the model function
* Calculate a covariance matrix for a variation of the systematic parameter
* Use the covariance to calculate the sensitivity impact compared to the statistical sensitivity in a contour plot

In [ ]:
import numpy as np
import plotInterface as pi; pi.init()
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.optimize import curve_fit
import scipy.stats as stats
from fastprogress.fastprogress import master_bar, progress_bar

from spectrum import *
from binning import *
from responses import *

### Settings


In [ ]:
# Statistics
measTime = 3*365*24*60*60   # 3 years
rate     = 3e8              # total rate at detector
nEvents  = rate*measTime    # expected dataset size ~10^16

# Binning
nBins       = 200
eBinEdges   = np.linspace(0,40000,nBins+1)
eBinCenters = (eBinEdges[:-1]+eBinEdges[1:])/2
eBins = binning.from_centers(eBinCenters)

print(f'Number of events: {nEvents:e}')

# Systematic parameter variation
wcc_val = 20e-6
wcc_err = wcc_val*0.10 # Assume 10% Uncertainty

### The model

In [ ]:
def model(eBins, amplitude, Epae=0, mSterile=0, sin2theta=0, wcc=wcc_val):
    
    # define function for bin integration
    spec = lambda e: diffspec_mixed(e-Epae, mSterile=mSterile, sin2theta=sin2theta)
    
    # integrate over bins
    binnedSpec = integrate_bins_fast(spec,eBins)
    
    # normalize to amplitude
    binnedSpec = amplitude*binnedSpec/binnedSpec.sum()
    
    # get response
    resp = calculate_cs_response(wcc,1e-3,1e-10,eBins,eBins)
    
    # apply response
    binnedSpec = np.dot(resp.T,binnedSpec)
    
    return binnedSpec

In [ ]:
# evaluate model once
yData = model(eBins, nEvents)

# plot as step histogram
plt.step(eBinCenters,yData)
plt.step(eBinCenters,model(eBins, nEvents, 0, 10000, 0.3))

pi.plotty()

### Covariance statistical error

In [ ]:
# calculate covairance matrix for statistical error
covStat = np.ones((nBins,nBins)) # initialize with ones to ensure inversibility
for i in range(nBins):
    covStat[i][i] = yData[i]

# for convenience also 1d error array for satstistics
yErr = np.sqrt(yData)

# show matrix
plt.imshow(covStat)
plt.colorbar()
pi.plotty()

### Covariance systematic error

In [ ]:
nExp = 10000

wccs = np.random.normal(wcc_val, wcc_err, size=nExp)

contents, edges = np.histogram(wccs*1e6,bins=100)
pi.plot_hist(edges,contents)
pi.plotty(xlabel=r'charge cloud width wcc ($\mu m$)')

In [ ]:
# use less nExp to make it faster
nExp = 100
wccs = np.random.normal(wcc_val, wcc_err, size=nExp)

# calculate many spectra, each with different value for systematic parameter
mcData = np.zeros((nBins,nExp))
for i in progress_bar(range(nExp)):
    mcData[:,i] = model(eBins, nEvents, 0, 0, 0, wccs[i])

plt.step(eBins.centers,mcData)
pi.plotty()

In [ ]:
# estimate covariance from MC spectra
covSys = np.cov(mcData)

# show matrix
plt.imshow(covSys, norm=colors.SymLogNorm(1e22,base=10))
plt.colorbar()
pi.plotty()

### Chisquare grid calculation 

In [ ]:
# define grid over mixing angles and sterile masses
mixings = np.logspace(-5,-8,10)
masses  = np.linspace(0,ENDPOINT,20)

# invert covariance matrix
covInvStat = np.linalg.inv(covStat)
covInvSys  = np.linalg.inv(covStat+covSys)

# calculate chisquare over grid
chisquareStat = np.zeros((len(mixings),len(masses)))
chisquareSys  = np.zeros((len(mixings),len(masses)))

# set up nested progress bar
mb = master_bar(range(len(masses)),total_time=True)
mb.main_bar.comment = 'ms'
pb = progress_bar(range(len(mixings)), parent=mb)
mb.child.comment = 's2t'

for i in mb:
    for j in pb:
        ms  = masses[i]
        s2t = mixings[j]
        
        # evealuate model for fixed parameters
        yModel = model(eBins, nEvents, mSterile=ms, sin2theta=s2t)
        
        # define local fit model function where normalisation can be varied
        def fitmodel(x,norm):    
            return norm*yModel
        
        # fit the normalisation
        par, cov = curve_fit(fitmodel,eBinCenters,yData,sigma=yErr,p0=[1.0])
        
        # evaluate fitmodel for fitted normalisation
        yFit = fitmodel(eBinCenters,*par)
        
        # calculate chisquare
        yResidual = yData - yFit        
        chisquareStat[j][i] = yResidual.dot(covInvStat.dot(yResidual))
        chisquareSys[j][i]  = yResidual.dot(covInvSys.dot(yResidual))

### Exclusion / Sensitivity plot

In [ ]:
# critical chisquare value for 90% confidence for 2 parameters (here: mass, mixing)
chiSquareCrit = stats.chi2.ppf(0.95, df=2)
print(f'Critical value for 95% confidence: {chiSquareCrit}')

# plot 90% exclusion contours
plt.contour(masses/1000, mixings, chisquareSys, 
            levels=[chiSquareCrit], colors='r')
plt.contour(masses/1000, mixings, chisquareStat, 
            levels=[chiSquareCrit], colors='k', linestyles='dashed',alpha=0.5)

# show plot with labels
pi.plotty(xlabel=r'$m_\mathrm{s}$ (keV)', ylabel=r'$\sin^2\theta$',log='y')

## With 10kV post acceleration

In [ ]:
# evaluate model once
yData = model(eBins, nEvents, Epae=10000)

# plot as step histogram
plt.step(eBinCenters,yData)
plt.step(eBinCenters,model(eBins, nEvents, 10000, 10000, 0.3))

pi.plotty()

In [ ]:
# calculate covairance matrix for statistical error
covStat = np.ones((nBins,nBins)) # initialize with ones to ensure inversibility
for i in range(nBins):
    covStat[i][i] = yData[i]

# for convenience also 1d error array for satstistics
yErr = np.sqrt(yData)

# use less nExp to make it faster
nExp = 100
wccs = np.random.normal(wcc_val, wcc_err, size=nExp)

# calculate many spectra, each with different value for systematic parameter
mcData = np.zeros((nBins,nExp))
for i in progress_bar(range(nExp)):
    mcData[:,i] = model(eBins, nEvents, 10000, 0, 0, wccs[i])

plt.step(eBins.centers,mcData)
pi.plotty()

# estimate covariance from MC spectra
covSys = np.cov(mcData)

In [ ]:
# define grid over mixing angles and sterile masses
mixings = np.logspace(-5,-8,10)
masses  = np.linspace(0,ENDPOINT,20)

# invert covariance matrix
covInvStat = np.linalg.inv(covStat)
covInvSys  = np.linalg.inv(covStat+covSys)

# calculate chisquare over grid
chisquareStat = np.zeros((len(mixings),len(masses)))
chisquareSys  = np.zeros((len(mixings),len(masses)))

# set up nested progress bar
mb = master_bar(range(len(masses)),total_time=True)
mb.main_bar.comment = 'ms'
pb = progress_bar(range(len(mixings)), parent=mb)
mb.child.comment = 's2t'

for i in mb:
    for j in pb:
        ms  = masses[i]
        s2t = mixings[j]
        
        # evealuate model for fixed parameters
        yModel = model(eBins, nEvents, mSterile=ms, sin2theta=s2t,Epae=10000)
        
        # define local fit model function where normalisation can be varied
        def fitmodel(x,norm):    
            return norm*yModel
        
        # fit the normalisation
        par, cov = curve_fit(fitmodel,eBinCenters,yData,sigma=yErr,p0=[1.0])
        
        # evaluate fitmodel for fitted normalisation
        yFit = fitmodel(eBinCenters,*par)
        
        # calculate chisquare
        yResidual = yData - yFit        
        chisquareStat[j][i] = yResidual.dot(covInvStat.dot(yResidual))
        chisquareSys[j][i]  = yResidual.dot(covInvSys.dot(yResidual))

In [ ]:
# critical chisquare value for 90% confidence for 2 parameters (here: mass, mixing)
chiSquareCrit = stats.chi2.ppf(0.95, df=2)
print(f'Critical value for 95% confidence: {chiSquareCrit}')

# plot 90% exclusion contours
plt.contour(masses/1000, mixings, chisquareSys, 
            levels=[chiSquareCrit], colors='r')
plt.contour(masses/1000, mixings, chisquareStat, 
            levels=[chiSquareCrit], colors='k', linestyles='dashed',alpha=0.5)

# show plot with labels
pi.plotty(xlabel=r'$m_\mathrm{s}$ (keV)', ylabel=r'$\sin^2\theta$',log='y')